<a href="https://colab.research.google.com/github/punit0087/Covid19India.Org-Tracker/blob/master/Covid19IndiaTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##Code written by Punit Rathore (prathore@mit.edu)
import numpy as np
import pandas as pd
import requests
from math import sin, cos, sqrt, atan2, radians

import warnings
warnings.filterwarnings("ignore")

In [0]:
#https://drive.google.com/file/d/1FBbr9zubK4VsB6gmNzj4fvE204ngvCeZ/view?usp=sharing
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

if __name__ == "__main__":
    file_id = '1FBbr9zubK4VsB6gmNzj4fvE204ngvCeZ'
    destination = 'IndiaPostalCodes.csv'
    download_file_from_google_drive(file_id, destination)

In [0]:

def get_distance_between_lats_lons(lat1,lon1,lat2,lon2):
# approximate radius of earth in km
        R = 6373

        lat1 = radians(lat1)
        lon1 = radians(lon1)
        lat2 = radians(lat2)
        lon2 = radians(lon2)

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c
        return(distance)

In [0]:
file= 'IndiaPostalCodes.csv'
city_wise_coordinates= pd.read_csv(file)
city_wise_coordinates['City'] = city_wise_coordinates['City'].str.upper()
city_wise_coordinates['District'] = city_wise_coordinates['District'].str.upper()
city_wise_coordinates['State'] = city_wise_coordinates['State'].str.upper()
#city_wise_coordinates

In [56]:
district_wise_pin_states= city_wise_coordinates.groupby('District')['PIN','State'].agg(lambda x:x.value_counts().index[0])
district_wise_lat_lng= city_wise_coordinates.groupby('District')['Lat','Lng'].agg(pd.Series.mean)

district_wise_data_geonames= district_wise_pin_states.merge(district_wise_lat_lng,left_on='District', right_on='District', how= 'inner').reset_index()
district_wise_data_geonames

,District,PIN,State,Lat,Lng
0,ADILABAD,504299,TELANGANA,19.175991,78.811514
1,AGRA,283111,UTTAR PRADESH,27.093774,78.185744
2,AHMADABAD,382150,GUJARAT,22.855290,72.484435
3,AHMADNAGAR,422605,MAHARASHTRA,19.331382,74.914309
4,AIZAWL,796261,MIZORAM,23.714925,92.829337
...,...,...,...,...,...
659,WOKHA,797111,NAGALAND,26.170592,94.258863
660,YADGIR,585214,KARNATAKA,16.689803,76.918597
661,YAMUNA NAGAR,135133,HARYANA,30.201141,77.250576
662,YAVATMAL,445304,MAHARASHTRA,20.129862,78.132447


In [57]:
df= pd.read_json("https://api.covid19india.org/raw_data.json")## data from covid19india.org
df3= []
for row in range(0,df.shape[0]):
    df1= df['raw_data'][row]
    df2=pd.DataFrame(df1.items()).set_index(0)
    df3.append(df2.T)
                
appended_data = pd.concat(df3, sort=False)
appended_data.replace(r'^\s*$', np.nan, regex=True, inplace = True) 
appended_data.rename(columns={'detectedcity':'City'}, inplace=True)
appended_data.rename(columns={'detecteddistrict':'District'}, inplace=True)
appended_data.rename(columns={'detectedstate':'State'}, inplace=True)

appended_data['City'] = appended_data['City'].str.upper()
appended_data['District'] = appended_data['District'].str.upper()
appended_data['State'] = appended_data['State'].str.upper()
appended_data= appended_data.dropna(thresh=3)
appended_data


,agebracket,backupnotes,contractedfromwhichpatientsuspected,currentstatus,dateannounced,City,District,State,estimatedonsetdate,gender,nationality,notes,patientnumber,source1,source2,source3,statecode,statepatientnumber,statuschangedate,typeoftransmission
1,20,Student from Wuhan,NaN,Recovered,30/01/2020,THRISSUR,THRISSUR,KERALA,NaN,F,India,Travelled from Wuhan,1,https://twitter.com/vijayanpinarayi/status/122...,https://weather.com/en-IN/india/news/news/2020...,NaN,KL,KL-TS-P1,14/02/2020,Imported
1,NaN,Student from Wuhan,NaN,Recovered,02/02/2020,ALAPPUZHA,ALAPPUZHA,KERALA,NaN,NaN,India,Travelled from Wuhan,2,https://www.indiatoday.in/india/story/kerala-r...,https://weather.com/en-IN/india/news/news/2020...,NaN,KL,KL-AL-P1,14/02/2020,Imported
1,NaN,Student from Wuhan,NaN,Recovered,03/02/2020,KASARAGOD,KASARAGOD,KERALA,NaN,NaN,India,Travelled from Wuhan,3,https://www.indiatoday.in/india/story/kerala-n...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,KL,KL-KS-P1,14/02/2020,Imported
1,45,Travel history to Italy and Austria,NaN,Recovered,02/03/2020,EAST DELHI (MAYUR VIHAR),EAST DELHI,DELHI,NaN,M,India,"Travelled from Austria, Italy",4,https://www.indiatoday.in/india/story/not-a-ja...,https://economictimes.indiatimes.com/news/poli...,NaN,DL,DL-P1,15/03/2020,Imported
1,24,"Travel history to Dubai, Singapore contact",NaN,Recovered,02/03/2020,HYDERABAD,HYDERABAD,TELANGANA,NaN,M,India,"Travelled from Dubai to Bangalore on 20th Feb,...",5,https://www.deccanherald.com/national/south/qu...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,TG,TS-P1,02/03/2020,Imported
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,NaN,NaN,NaN,Hospitalized,03/04/2020,NaN,NaN,UTTAR PRADESH,NaN,NaN,NaN,NaN,3104,State Bulletin,NaN,NaN,UP,NaN,03/04/2020,NaN
1,NaN,NaN,NaN,Hospitalized,03/04/2020,NaN,NaN,UTTAR PRADESH,NaN,NaN,NaN,NaN,3105,State Bulletin,NaN,NaN,UP,NaN,03/04/2020,NaN
1,NaN,NaN,E0,Hospitalized,03/04/2020,NaN,KAMRUP,ASSAM,NaN,NaN,NaN,Attended Delhi Religious Conference,3106,https://twitter.com/ANI/status/124613704767870...,NaN,NaN,NaN,NaN,03/04/2020,NaN
1,NaN,NaN,E0,Hospitalized,03/04/2020,NaN,KAMRUP METROPOLITAN,ASSAM,NaN,NaN,NaN,Attended Delhi Religious Conference,3107,https://twitter.com/ANI/status/124613704767870...,NaN,NaN,NaN,NaN,03/04/2020,NaN


In [58]:
district_wise_counts= appended_data.groupby('District').agg({'patientnumber': 'count'})
district_wise_counts.rename(columns={'patientnumber':'d_patient_counts'}, inplace=True)
district_wise_counts =district_wise_counts.reset_index()
district_wise_counts['District'] = district_wise_counts['District'].str.upper()
district_wise_counts

,District,d_patient_counts
0,AGRA,9
1,AHMADABAD,38
2,AHMADNAGAR,19
3,AIZAWL,1
4,AJMER,5
...,...,...
218,WAYANAD,3
219,WEST DELHI,2
220,WEST GODAVARI,15
221,Y.S.R.,19


In [59]:
corona_db_with_latlng= district_wise_counts.merge(district_wise_data_geonames, left_on='District', right_on='District', how= 'inner')
corona_db_with_latlng.rename(columns={'d_patient_counts':'Num_Positive_cases'}, inplace=True)
corona_db_with_latlng

,District,Num_Positive_cases,PIN,State,Lat,Lng
0,AGRA,9,283111,UTTAR PRADESH,27.093774,78.185744
1,AHMADABAD,38,382150,GUJARAT,22.855290,72.484435
2,AHMADNAGAR,19,422605,MAHARASHTRA,19.331382,74.914309
3,AIZAWL,1,796261,MIZORAM,23.714925,92.829337
4,AJMER,5,305801,RAJASTHAN,26.240941,74.715133
...,...,...,...,...,...,...
192,VISAKHAPATNAM,15,531040,ANDHRA PRADESH,17.834689,82.843291
193,WAYANAD,3,673592,KERALA,11.657758,76.081974
194,WEST DELHI,2,110018,DELHI,28.632937,77.132149
195,WEST GODAVARI,15,534455,ANDHRA PRADESH,16.826023,81.393287


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
def get_idx_distance_from_query_locations(q_lat, q_lng, corona_db_with_latlng):
    dist_array=[]
    for index, row in corona_db_with_latlng.iterrows():
        dist= int(get_distance_between_lats_lons(q_lat,q_lng,row['Lat'],row['Lng']))
        dist_array.append(dist)
    
    minpos = dist_array.index(min(dist_array)) 
    mindist= dist_array[minpos]
    cases= corona_db_with_latlng.loc[minpos,'Num_Positive_cases']
    location= corona_db_with_latlng.loc[minpos,'District']
    state= corona_db_with_latlng.loc[minpos,'State']
    Lats= corona_db_with_latlng.loc[minpos,'Lat']
    Lngs= corona_db_with_latlng.loc[minpos,'Lng']

    return(mindist, cases, location, state)

In [0]:
def get_nearest_covid19_stats(query_info,corona_db_with_latlng):
    if query_info.PIN.iloc[1] in corona_db_with_latlng['PIN'].values:
        mindist= 2
        Lat= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'Lat'];
        Lng= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'Lng'];
        mindist= int(get_distance_between_lats_lons(query_info.Lat.iloc[1] ,query_info.Lng.iloc[1], Lat,Lng))
        cases= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'Num_Positive_cases']
        district= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'District']
        state= corona_db_with_latlng.loc[corona_db_with_latlng.PIN==query_info.PIN.iloc[1], 'State']
        print("The nearest location with COVID-19 from your PIN is in your own Postal Location with {} number of positive cases".format(cases.values[0]))
        print("Location: {} , {}".format(district.values[0].upper(), state.values[0].upper()))
    else:
        (mindist, cases, district, state) = get_idx_distance_from_query_locations(query_info.Lat.iloc[1] ,query_info.Lng.iloc[1], corona_db_with_latlng)  
        print("The nearest location with COVID-19 from your PIN is within {} km with {} number of positive cases".format(mindist, cases))
        print("Location: {} , {}".format(district.upper(), state.upper()))

In [63]:
query_pincode = input("Enter your location as an Indian PIN  : ") 
g = int(query_pincode)

if g in city_wise_coordinates.PIN.values:
    query_info= city_wise_coordinates[city_wise_coordinates.PIN == int(g)]
    get_nearest_covid19_stats(query_info,corona_db_with_latlng)
else:
    print('You entered an Invalid PIN')

Enter your location as an Indian PIN  : 825301
The nearest location with COVID-19 from your PIN is within 12 km with 1 number of positive cases
Location: HAZARIBAGH , JHARKHAND
